# Formatting

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html
# https://xlsxwriter.readthedocs.io/
# https://xlsxwriter.readthedocs.io/format.html
# https://xlsxwriter.readthedocs.io/working_with_conditional_formats.html

In [ ]:
import pandas as pd

In [ ]:
sales_data = pd.read_excel("sales.xlsx")

# Some inline formatting: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.style.html
# Heads up! Inline formatting will be included in later exports to excel.
## Better practice to perform data management and processing using pandas, and leve formatting to when writing to excel. 
#sales_data = sales_data.style.set_properties(**{'background-color': 'magenta', 'color': 'yellow'})

sales_data

In [ ]:
## Named sheets and basic formatting
# Requires a wee bit knowledge of CSS -> https://www.w3schools.com/css/css_intro.asp

dummy_data_df = pd.DataFrame({
    "A": [5, 6, 7],
    "B": ["x", "y", "z"]
})

with pd.ExcelWriter("formatted_sales.xlsx") as writer:
    
    # Add the separate sheets
    sales_data.to_excel(writer, sheet_name="Members", index=None)
    dummy_data_df.to_excel(writer, sheet_name="Dummy Data", index=None)
    
    ### Some basic formatting
    
    # Get reference to the sheets in workbook
    workbook = writer.book
    members_worksheet = writer.sheets["Members"]
    dummy_worksheet = writer.sheets["Dummy Data"]
        
    # Set width and bolden first column
    bold_cell_format = workbook.add_format({'bold': True})
    # Alternativly
    #bold_cell_format = workbook.add_format()
    #bold_cell_format.set_bold()
    members_worksheet.set_column(0, 0, width=30, cell_format=bold_cell_format)

    # Change format of numbers
    dollar_format = workbook.add_format({"num_format": "$#,##0.00"})
    #dummy_worksheet.set_column(0, 0, None, dollar_format)
    
    # Color a row
    row_color = workbook.add_format({"bg_color": "yellow"})
    #dummy_worksheet.set_row(2, cell_format=row_color)
    
    # Set format based on condition
    dummy_worksheet.conditional_format("A2:A4", {"type": "cell",
                                                 "criteria": ">",
                                                 "value": 5,
                                                 "format": row_color
                                                })


In [ ]:
# See https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html
## Heads up! From doc: "xlsxwriter for xlsx files if xlsxwriter is installed otherwise openpyxl"
## mode="a" not supported with xlsxwriter!
## I have xslxwriter installed, so must explicity set engine="openpyxl"

## Append to file (as new sheet)
## if_sheet_exists="overaly" -> writes to sheet from top down, overwriting any content
with pd.ExcelWriter("formatted_sales.xlsx", mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
    sales_data.to_excel(writer, sheet_name="New Sales", index=None)

# What are the different options of "if_sheet_exists"?
## if_sheet_exists{‘error’, ‘new’, ‘replace’, ‘overlay’}, default ‘error’
## error: raise a ValueError.
## new: Create a new sheet, with a name determined by the engine. (see example in last cell)
## replace: Delete the contents of the sheet before writing to it.
## overlay: Write contents to the existing sheet without removing the old contents.

In [ ]:
new_sales = pd.DataFrame({
    "Order ID": ["M0009"],
    "Product Name": ["Fish&Chips"],
    "Quantity": [2]
})

# Add new content to existing sheet, append and overaly, starting at startrow=15
with pd.ExcelWriter("formatted_sales.xlsx", mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
    new_sales.to_excel(writer, sheet_name="New Sales", startrow=15, header=None, index=None)

In [ ]:
# Add new content to a sheet with name "New Sales", see if_sheet_exists="new"
## If sheet exists, create new -> will auto append a number at end of name. If name ends with number, will increment by 1.
with pd.ExcelWriter("formatted_sales.xlsx", mode="a", engine="openpyxl", if_sheet_exists="new") as writer:
    new_sales.to_excel(writer, sheet_name="New Sales", index=None)

# Every time this cell is run, a new sheet is added to the workbook, named "New Sales1", "New Sales2", "new Sales3" etc...